In [ ]:
%pip install qdrant_client

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import torch
from urllib.parse import urlparse


In [ ]:
qdranturl = 'qdrant://localhost:6333#test_qdrant'

In [ ]:
o = urlparse(qdranturl, allow_fragments=True)

In [ ]:
print(
    o.scheme,
    o.hostname,
    o.port,
    o.path,
    o.fragment,
)

In [ ]:
client = QdrantClient(host=o.hostname, port=o.port, grpc_port=(o.port+1), prefer_grpc=True)

In [ ]:
try:
    client.get_collections()[0]
    print("Qdrant server is alive and running")
except Exception as e:
    print(f"Error connecting to Qdrant server: {e}")

In [ ]:
if client.collection_exists(collection_name=o.fragment):
    print('collection exists')
else:
    print('collection does not exist')
    client.create_collection(
        collection_name=o.fragment,
        vectors_config=models.VectorParams(size=5, distance=models.Distance.COSINE),
    )

collection_info = client.get_collection(collection_name=o.fragment)
print(list(collection_info))

# PUT /collections/benchmark 
# {
#   "vectors": {
#     ...
#     "on_disk": true
#   },
#   "hnsw_config": {
#     "on_disk": true
#   },
#   ...
# }


In [ ]:
a = torch.rand(int(1e4),5)
print(a.size())

In [ ]:
## add half of data
operation_info = client.upsert(
    collection_name=o.fragment,
    wait=True,
    points=[models.PointStruct(id=i, vector=a[i]) for i in range(int(5e3))]
)
print(operation_info)



In [ ]:
# add other half of data batched
operation_info = client.upsert(
    collection_name=o.fragment,
    wait=True,
    points=models.Batch(
        ids=list(range(int(5e3), int(1e4))),
        vectors=a[int(5e3):]
    )
)
print(operation_info)

In [ ]:
result = client.retrieve(
    collection_name=o.fragment,
    ids=[12],
    with_vectors=True,
    with_payload=False
)

In [ ]:
print(len(result))
print(result[0].id)
print(torch.tensor(result[0].vector, dtype=torch.float32))

In [ ]:
collection_info = client.get_collection(collection_name=o.fragment)
print(collection_info)